In [1]:
%load_ext lineapy

In [2]:
import sys
import importlib
import json
import datetime
print('current UTC:', datetime.datetime.utcnow())

current UTC: 2022-10-13 02:13:03.289769


In [3]:
cpp_config_path = '../config/option_config.json'
params = json.load(open(cpp_config_path,'r'))
lineapy.save(params, 'quantlib_config')
print(params)

{'option_type': 'call', 'num_sims': 1000, 'S': 100, 'K': 100, 'r': 0.01, 'v': 0.25, 'T': 1}


In [4]:
def load_quantlib(version='1.25'):
    sys.path.append(f"/home/mingjerli/quantlib/QuantLib-{version}")
    import QuantLib
    print(f'Loading QuantLib v{QuantLib.__version__}')
    return QuantLib

In [5]:
version = '1.27'
ql = load_quantlib(version)
ver = str(ql.__version__)
art = lineapy.save(ver,'quantlib_version')
print(art.get_code())


Loading QuantLib v1.27
import sys


def load_quantlib(version="1.25"):
    sys.path.append(f"/home/mingjerli/quantlib/QuantLib-{version}")
    import QuantLib

    print(f"Loading QuantLib v{QuantLib.__version__}")
    return QuantLib


version = "1.27"
ql = load_quantlib(version)
ver = str(ql.__version__)



In [6]:
today = datetime.datetime.today()
maturity_date = today + datetime.timedelta(365*params['T'])
calculation_date = ql.Date(today.day,  today.month, today.year +1)
maturity_date = ql.Date(maturity_date.day,  maturity_date.month, maturity_date.year +1)

In [7]:
def quantlib_european_call(
    maturity_date = maturity_date,
    calculation_date = calculation_date,
    spot_price = params['S'],
    strike_price = params['K'],
    volatility = params['v'],
    risk_free_rate = params['r']
):
    dividend_rate =  0
    option_type = ql.Option.Call

    day_count = ql.Actual365Fixed()
    calendar = ql.UnitedStates()

    ql.Settings.instance().evaluationDate = calculation_date    

    # construct the European Option
    payoff = ql.PlainVanillaPayoff(option_type, strike_price)
    exercise = ql.EuropeanExercise(maturity_date)
    european_option = ql.VanillaOption(payoff, exercise)

    # Setup Pricing Engine
    spot_handle = ql.QuoteHandle(
        ql.SimpleQuote(spot_price)
    )
    flat_ts = ql.YieldTermStructureHandle(
        ql.FlatForward(calculation_date, risk_free_rate, day_count)
    )
    dividend_yield = ql.YieldTermStructureHandle(
        ql.FlatForward(calculation_date, dividend_rate, day_count)
    )
    flat_vol_ts = ql.BlackVolTermStructureHandle(
        ql.BlackConstantVol(calculation_date, calendar, volatility, day_count)
    )
    bsm_process = ql.BlackScholesMertonProcess(spot_handle, 
                                               dividend_yield, 
                                               flat_ts, 
                                               flat_vol_ts)

    european_option.setPricingEngine(
        ql.AnalyticEuropeanEngine(bsm_process)
    )
    bs_price = european_option.NPV()
    return bs_price

In [8]:
bs_price = quantlib_european_call(
    maturity_date = maturity_date,
    calculation_date = calculation_date,
    spot_price = params['S'],
    strike_price = params['K'],
    volatility = params['v'],
    risk_free_rate = params['r']
)

In [9]:
bs_price

10.418271653959474

In [10]:
quantlib_bs_price = lineapy.save(bs_price,'quantlib_price')